<a href="https://colab.research.google.com/github/LittleDarwin2021/mediapipe_get_coordinate/blob/main/mediapipe_for_movie20230301.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
!pip install mediapipe

In [2]:
import glob
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
import os
import shutil

In [3]:
!mkdir "save_imgs"
!mkdir "load_imgs"
!mkdir "save_movie1"
!mkdir "save_movie2"
!mkdir "load_movie"

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [5]:
#静止画の座標を取る関数
def create_table(image_num, results, image_height, image_width):
  x_list = []
  y_list = []
  z_list = []
  results_list = results.pose_landmarks.landmark
  for i, result in enumerate(results_list) :
    x_list.append(result.x)
    y_list.append(result.y)
    z_list.append(result.z)
  all_list = [image_num] + x_list + y_list + z_list + [image_height] +[image_width]
  return all_list

# ビデオの読み込み＋静止画に分解

In [10]:
#動画ファイルの指定
video_path = "load_movie/sample3.mov"

In [11]:
#動画ファイルを画像に分解
#分解された画像ファイル群はsave_movie1フォルダに保存

if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= video_path,   
                   image_dir='save_movie1/', 
                   image_file='%s.png'):
 
    
    i = 0
    #フレームをスキップしたい場合は2以上の数値を入力
    interval = 1
    length = 300
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

# MediaPipeで静止画を処理

In [12]:
#座標を格納するリスト
coordinate = []

IMAGE_FILES = []
for name in sorted(glob.glob('save_movie1/*.png')):
    IMAGE_FILES.append(name)

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  #元々動画だったものを1枚ずつ画像として骨格推定の処理を行う
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    #元画像のサイズを抽出する。座標データは正規化処理されるので、元のアスペクト比に戻したい時にこれらを使う
    image_height, image_width, _ = image.shape
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    
    if not results.pose_landmarks:
      continue
    

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    #座標データを抽出する
    all_list = create_table(idx, results, image_height, image_width)
    coordinate.append(all_list)
    #加工された画像はsave_movie2フォルダに保存
    cv2.imwrite('save_movie2/' + str(idx).zfill(6) + '.png', image)


In [13]:
#pandasカラム名の準備
x_num, y_num, z_num = [], [], []
for i in range(33):
  x_num.append("x" +"{}".format(i).zfill(2))
  y_num.append("y" +"{}".format(i).zfill(2))
  z_num.append("z" +"{}".format(i).zfill(2))
column_list = ["num"] +x_num + y_num + z_num + ["image_height"] + ["image_width"]

In [14]:
#座標情報をデータフレーム化する
df = pd.DataFrame(coordinate, columns = column_list)

# 処理した画像をmp4動画に変換


In [ ]:
# 既に output.mp4 があれば削除
if os.path.exists('output.mp4'):
   os.remove('output.mp4')

! ffmpeg -r 10 -i save_movie2/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")